# Production optimisation with ensembles – an interactive tutorial

Copyright Patrick N. Raanes, NORCE, 2023.

This is a self-contained tutorial on production optimisation using ensemble methods.
- Please have a look at the [history matching (HM) tutorial](MAIN.ipynb)
  for an introduction to Python, Jupyter notebooks, and this reservoir simulator.

If you're on **Google Colab**, run the cell below to install the requirements.
Otherwise (and assuming you have done the installation described in the README),
you can skip/delete this cell.

# TODO:
- Use Bezier curves to parametrize well rates ?
- Make user widget for manual optimisation
- 1D: Plot ensemble of npv curves
- Plot ensemble of pdfs of npvs, including
  strategies: reactive control, nominal optimization, robust optimization
- Use model with many injectors, producers ? jansen2010closed_prez
- Cite essen, jansen, chen, fonseca, stordal, raanes

In [ ]:
## Imports
remote = "https://raw.githubusercontent.com/patnr/HistoryMatching"
!wget -qO- {remote}/master/colab_bootstrap.sh | bash -s

In [ ]:
from dataclasses import dataclass

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from tools import mpl_setup

In [ ]:
mpl_setup.init()
from tqdm.auto import tqdm

In [ ]:
plt.ion()
np.set_printoptions(precision=6)

In [ ]:
%matplotlib inline

In [ ]:
import copy

In [ ]:
import numpy.random as rnd
import TPFA_ResSim as simulator
# from matplotlib.ticker import LogLocator
from mpl_tools.place import freshfig
# from numpy import sqrt
from struct_tools import DotDict as Dict

In [ ]:
import tools.plotting as plotting
# import tools.localization as loc
from tools import geostat, utils
from tools.utils import apply

from tools.utils import center

In [ ]:
## Aux
def final_sweep(title="", **kwargs):
    model = model_setup(**kwargs)
    wsats, prods = sim(model, wsat0)
    plotting.single.model = model
    fig, ax = freshfig(f"Final sweep {title}", figsize=(1, .6), rel=True)
    plotting.single.field(ax, wsats[-1], "oil", wells=True, colorbar=True)
    fig.tight_layout()

In [ ]:
# Could put this in npv() rather than copying the base model
# but it's useful to have a base model in the global namespace
# for plotting purposes.
perm = Dict()

In [ ]:
def sample_prior_perm(N):
    lperms = geostat.gaussian_fields(model.mesh, N, r=0.8)
    return lperms

In [ ]:
def perm_transf(x):
    return .1 + np.exp(5*x)

In [ ]:
plotting.styles["pperm"]["levels"] = np.linspace(-4, 4, 21)
plotting.styles["pperm"]["cticks"] = np.arange(-4, 4+1)

In [ ]:
def set_perm(model, log_perm_array):
    """Set perm. in model code (both x and y components)."""
    p = perm_transf(log_perm_array)
    p = p.reshape(model.shape)
    model.Gridded.K = np.stack([p, p])

In [ ]:
@dataclass
class Momentum:
    """Gradient momentum (provides history memorisation)."""

    # Note: I also tried rolling average with a maxlen deque
    #       but the results of cursory testing were slightly worse.
    b: float = 0.9

    def update(self, v):
        if not hasattr(self, 'val'):
            self.reset()
        self.val *= self.b
        self.val += (1-self.b) * v

    def reset(self, val=0):
        self.val = val

In [ ]:
def EnGrad(obj, u, chol, precond=False):
    U, _ = utils.center(rnd.randn(N, len(u)) @ chol.T)
    J, _ = utils.center(obj(u + U))
    if precond:
        g = U.T @ J / (N-1)
    else:
        g = utils.rinv(U, reg=.1, tikh=True) @ J
    return g

In [ ]:
## EnOpt
def EnOpt(obj, u, chol, sign=+1,
          # Step modifiers:
          regulator=None, xSteps=(1,), normed=True, precond=True,
          # Stopping criteria:
          nIter=100, rtol=1e-4):
    """Gradient/steepest *descent* using ensemble (LLS) gradient and backtracking.

    - `rtol` specified how large an improvement is required to update the iterate.
      Large values makes backtracking more reluctant to accept an update,
      resulting in *faster* declaration of convergence.
      Setting to 0 is not recommended, because if the objective function is flat
      in the neighborhood, then the path could just go in circles on that flat.
    """

    def backtrack(base_step):
        """Line search by bisection."""
        with tqdm(total=len(xSteps), desc="Backtrack", leave=False) as pbar:
            for i, xStep in enumerate(xSteps):
                pbar.update(1)
                x = path[-1] + sign * xStep * base_step
                J = obj(x)
                if sign*(J - objs[-1]) > atol:
                    pbar.update(len(xSteps))  # needed in Jupyter
                    return x, J, i

    # Init
    if regulator:
        regulator.reset()
    J = obj(u)
    atol = max(1e-8, abs(J)) * rtol
    info, path, objs = {}, [], []

    for itr in range(nIter):
        path.append(u)
        objs.append(J)

        # Compute search direction
        grad = EnGrad(obj, u, chol, precond=precond)
        if normed:
            grad /= utils.mnorm(grad)
        if regulator:
            regulator.update(grad)

        # Update iterate
        if not regulator or not (updated := backtrack(regulator.val)):
            if regulator:
                regulator.reset(grad)
                info.setdefault('resets', []).append(itr)
            # Fallback to pure grad
            if not (updated := backtrack(grad)):
                # Stop if lower J not found
                status = "Converged ✅"
                break
        u, J, i = updated
        info.setdefault('nDeclined', []).append(i)

    else:
        status = "Ran out of iters ❌"

    print(f"{status:<9} {itr=:<5}  {path[-1]=}  {objs[-1]=:.2f}")
    return np.array(path), np.array(objs), info

In [ ]:
# ## Model case
def model_setup(**kwargs):
    """Create new model, based on `globals()['model']`."""
    # Init
    modln = copy.deepcopy(model)  # dont overwrite
    # pperm gets special treatment (transformation)
    set_perm(modln, kwargs.pop('perm', pperm))
    # Set other attrs
    for key, val in kwargs.items():
        setattr(modln, key, val)
    # Sanitize
    modln.config_wells(modln.inj_xy, modln.inj_rates,
                       modln.prod_xy, modln.prod_rates)
    return modln

In [ ]:
model = simulator.ResSim(Nx=20, Ny=20, Lx=2, Ly=1)
seed = rnd.seed(3)
pperm = sample_prior_perm(1)

In [ ]:
xy_4corners = np.dstack(np.meshgrid(
    np.array([.12, .87]) * model.Lx,
    np.array([.12, .87]) * model.Ly,
)).reshape((-1, 2))

In [ ]:
r0 = 1.5
model = model_setup(
    inj_xy = np.array([model.domain[1]]) / 2,
    inj_rates = r0 * np.ones((1, 1)) / 1,
    prod_rates = r0 * np.ones((4, 1)) / 4,
    prod_xy = xy_4corners,
)
plotting.single.model = model
plotting.single.coord_type = "absolute"

In [ ]:
# Simulation
wsat0 = np.zeros(model.Nxy)
T = 1
dt = 0.025
nTime = round(T/dt)

In [ ]:
# Like `forward_model`, but w/o setting params
def sim(model, wsat, pbar=False, leave=False):
    """Simulate reservoir."""
    integrator = model.time_stepper(dt)
    wsats = simulator.recurse(integrator, nTime, wsat, pbar=pbar, leave=leave)
    # Extract production time series from water saturation fields
    wells = model.xy2ind(*model.prod_xy.T)
    prods = np.array([wsat[wells] for wsat in wsats])
    return wsats, prods

In [ ]:
## Objectives
def prod2npv(model, prods):
    """Net present value (NPV), i.e. monetarily discounted, total oil production."""
    discounts = .99 ** np.arange(nTime + 1)
    prods = 1 - prods                   # water --> oil
    prods = prods * model.prod_rates.T  # volume = saturation * rate
    prods = np.sum(prods, -1)           # sum over wells
    value = prods @ discounts           # sum in time, incld. discount factors
    # Compute cost of water injection
    # PS: We don't bother with cost of water production,
    # since it is implicitly approximated by reduction in oil production.
    inj_rates = model.inj_rates
    if inj_rates.shape[1] == 1:
        inj_rates = np.tile(inj_rates, (1, nTime))
    cost = np.sum(inj_rates, 0)
    cost = cost @ discounts[:-1]
    return value - .5*cost

In [ ]:
def npv(**kwargs):
    """NPV from model config."""
    # Config
    try:
        model = model_setup(**kwargs)
        # Simulate
        wsats, prods = sim(model, wsat0)
    except Exception:
        return 0  # Invalid model params. Penalize. Use `raise` for debugging.
    return prod2npv(model, prods)

In [ ]:
def npv_in_injectors(xys):
    """`npv(inj_xy)`. Input shape `(nEns, 2*nInj)`."""
    xys, singleton = utils.atleast_2d(xys)
    xys = xys.reshape((len(xys), -1, 2))  # (nEns, 2*nInj) --> (nEns, nInj, 2)
    Js = apply(npv, inj_xy=xys, unzip=False,
               pbar=not singleton, leave=False)
    return Js[0] if singleton else Js

In [ ]:
def npv_in_x_of_inj0_with_fixed_y(x):
    """Like `npv_in_injectors` but with `y` fixed. Input shape `(nEns, 1)` or `(1,)`."""
    xs = x
    ys = y * np.ones_like(xs)
    xys = np.hstack([xs, ys])
    Js = npv_in_injectors(xys)
    return Js

obj = npv_in_x_of_inj0_with_fixed_y
y = model.Ly/2

In [ ]:
## Optim params
N = 10
xSteps = [.4 * 1/2**i for i in range(8)]
utils.nCPU = True

In [ ]:
# Plot objective
fig, ax = freshfig(f"{obj.__name__}({y})", figsize=(7, 3))
ax.set(xlabel="x", ylabel="NPV")
xx = np.linspace(0, model.Lx, 201)
npvs = obj(np.atleast_2d(xx).T)
ax.plot(xx, npvs, "slategrey", lw=3)

# Optimize
L = .3 * np.eye(1)
shifts = {}
u0s = model.Lx * np.array([[.05, .1, .2, .8, .9, .95]]).T
for i, u0 in enumerate(u0s):
    path, objs, info = EnOpt(obj, u0, L, regulator=None, xSteps=xSteps)
    shift = .3*i  # for visual distinction
    ax.plot(path, objs - shift, '-o', c=f'C{i+1}')
fig.tight_layout()

In [ ]:
obj = npv_in_injectors
fig, axs = plotting.figure12(obj.__name__)

# Plot objective
X, Y = model.mesh
XY = np.vstack([X.ravel(), Y.ravel()]).T
npvs = npv_in_injectors(XY)
plotting.field(axs[0], npvs, "obj", wells=True, argmax=True, colorbar=True);

# Optimize
# - Some get stuck in local minima
# - Smaller steps towards end
# - Notice that discreteness of npv_in_injectors function means that
#   they dont all CV to exactly the same
L = .1 * np.eye(2)
for color in ['C0', 'C2', 'C6', 'C7', 'C8', 'C9']:
    u0 = rnd.rand(2) * model.domain[1]
    path, objs, info = EnOpt(obj, u0, L, regulator=Momentum(0.3), xSteps=xSteps, precond=False)
    plotting.add_path12(*axs, path, objs, color=color)
fig.tight_layout()

In [ ]:
final_sweep("npv_in_injectors", inj_xy=path[-1].reshape((-1, 2)))

In [ ]:
def transform_xys(xys):
    """Transform infinite plane to `(0, Lx) x (0, Ly)`."""
    xys = np.array(xys, dtype=float)

    def realline_to_0L(x, L, compress=1):
        sigmoid = lambda z: 1/(1 + np.exp(-z))  # noqa
        x = (x - L/2) * L * compress
        return L * sigmoid(x)

    # Loop over (x, y)
    for i0, L in zip([0, 1], model.domain[1]):
        ii = slice(i0, None, 2)
        xys[..., ii] = realline_to_0L(xys[..., ii], L, 1)
    return xys

In [ ]:
def npv_in_injectors_transformed(xys):
    """Like `npv_in_injectors` but with transformation of (x, y)."""
    xys = transform_xys(xys)
    Js = npv_in_injectors(xys)
    return Js

In [ ]:
## Optimize 2 inj_xy
u0 = np.array([0, 0] + [2, 0])
model = model_setup(
    prod_xy = xy_4corners[:2],
    inj_xy = transform_xys(u0).reshape((2, 2)),
    prod_rates = r0 * np.ones((2, 1)) / 2,
    inj_rates = r0 * np.ones((2, 1)) / 2,
)
plotting.single.model = model
fig, axs = plotting.figure12("npv_in_injectors_transformed")
plotting.field(axs[0], pperm, "pperm", wells=True, colorbar=True)

L = .1 * np.eye(len(u0))
path, objs, info = EnOpt(npv_in_injectors_transformed, u0, L,
                         regulator=Momentum(.1), xSteps=xSteps, precond=False, rtol=1e-8)
path = transform_xys(path)

plotting.add_path12(*axs, path[:, :2], objs, color='C0')
plotting.add_path12(*axs, path[:, 2:], color='C5')
fig.tight_layout()

In [ ]:
final_sweep("npv_in_injectors_transformed", inj_xy=path[-1].reshape((-1, 2)))

In [ ]:
## Rates
def npv_in_rates(inj_rates):
    """`npv(inj_rates)`. Input shape `(nEns, nInj)`."""
    inj_rates, singleton = utils.atleast_2d(inj_rates)
    nEns = len(inj_rates)
    inj_rates = inj_rates.reshape((nEns, -1, 1))  # (nEns, nInj) --> (nEns, nInj, 1)
    total_rate = np.sum(inj_rates, axis=1).squeeze()  # (nEns,)
    nProd = len(model.prod_rates)
    prod_rates = (np.ones((1, nProd, nEns)) * total_rate/nProd).T
    Js = apply(npv, inj_rates=inj_rates, prod_rates=prod_rates, unzip=False,
               pbar=not singleton, leave=False)
    return Js[0] if singleton else Js

In [ ]:
# Restore default well config
model = model_setup(
    inj_xy = np.array([model.domain[1]]) / 2,
    inj_rates = r0 * np.ones((1, 1)) / 1,
    prod_rates = r0 * np.ones((4, 1)) / 4,
    prod_xy = xy_4corners,
)
plotting.single.model = model

In [ ]:
obj = npv_in_rates

xx = np.linspace(0.1, 5, 21)
npvs = obj(np.atleast_2d(xx).T)

fig, ax = freshfig(obj.__name__)
ax.grid()
ax.set(xlabel="rate", ylabel="NPV")
ax.plot(xx, npvs, "slategrey")

for i, u0 in enumerate(np.array([[.1, 5]]).T):
    L = .1 * np.eye(len(u0))
    path, objs, info = EnOpt(obj, u0, L, xSteps=xSteps, precond=False, rtol=1e-8)
    shift = i+1  # for visual distinction
    ax.plot(path, objs - shift, '-o', color=f'C{i+1}')

# Triangles

In [ ]:
triangle = [0, 135, -135]
wells = dict(
    inj_xy = ([[model.Lx/2, model.Ly/2]] +
              [utils.xy_p_normed(th, *model.domain[1]) for th in +90 + triangle]),
    prod_xy = [utils.xy_p_normed(th, *model.domain[1]) for th in -90 + triangle],
    inj_rates = r0 * np.ones((4, 1)) / 4,
    prod_rates = r0 * np.ones((3, 1)) / 3,
)

# fig, ax = freshfig("Triangles")
model = model_setup(**wells)
plotting.single.model = model
# plotting.field(ax, pperm, "pperm", wells=True, colorbar=True)

In [ ]:
def final_sweep2(i0, i1, i2, i3):
    inj_rates = np.array([[i0, i1, i2, i3]]).T
    prod_rates = np.ones((3, 1)) / 3 * np.sum(inj_rates)
    model = model_setup(inj_rates=inj_rates, prod_rates=prod_rates)
    plotting.single.model = model
    wsats, prods = sim(model, wsat0)
    npv = prod2npv(model, prods)
    print(f"NPV: {npv:.5f}")
    return wsats[-1]
    # fig, ax = freshfig(f"Final sweep {title}", figsize=(1, .6), rel=True)
    # plotting.single.field(ax, wsats[-1], "oil", wells=True, colorbar=True)
    # fig.tight_layout()

In [ ]:
final_sweep2.controls = dict(
    i0 = (0, 1.9),
    i1 = (0, 1.9),
    i2 = (0, 1.9),
    i3 = (0, 1.9),
)

In [ ]:
plotting.field_console(final_sweep2, "oil", wells=True)

In [ ]:
## Optim triangles
u0 = .7*np.ones(len(model.inj_rates))
L = .1 * np.eye(len(u0))
path, objs, info = EnOpt(npv_in_rates, u0, L, xSteps=xSteps, precond=False, rtol=1e-8)
# final_sweep("Triangles", **wells)